task name: Resolve MapValue Task

task description: Implement the provided C# code as a dotnet interactive notebook.

task name: Resolve MapValue Task
task description: Implement the provided C# code as a dotnet interactive notebook.

In [ ]:
#r "nuget:Microsoft.ML, 1.5.2"

In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;

In [ ]:
public class DataPoint
{
    public string Timeframe { get; set; }
    public int Score { get; set; }
}

var mlContext = new MLContext();
var rawData = new[] {
    new DataPoint() { Timeframe = "0-4yrs" , Score = 1 },
    new DataPoint() { Timeframe = "6-11yrs" , Score = 2 },
    new DataPoint() { Timeframe = "12-25yrs" , Score = 3 },
    new DataPoint() { Timeframe = "0-5yrs" , Score = 4 },
    new DataPoint() { Timeframe = "12-25yrs" , Score = 5 },
    new DataPoint() { Timeframe = "25+yrs" , Score = 5 },
};
var data = mlContext.Data.LoadFromEnumerable(rawData);

In [ ]:
var timeframeMap = new Dictionary<string, string>();
timeframeMap["0-4yrs"] = "Short";
timeframeMap["0-5yrs"] = "Short";
timeframeMap["6-11yrs"] = "Medium";
timeframeMap["12-25yrs"] = "Long";
timeframeMap["25+yrs"] = "Long";

var timeframeKeyMap = new Dictionary<string, uint>();
timeframeKeyMap["0-4yrs"] = 1;
timeframeKeyMap["0-5yrs"] = 1;
timeframeKeyMap["6-11yrs"] = 2;
timeframeKeyMap["12-25yrs"] = 3;
timeframeKeyMap["25+yrs"] = 3;

var scoreMap = new Dictionary<int, string>();
scoreMap[1] = "Low";
scoreMap[2] = "Low";
scoreMap[3] = "Average";
scoreMap[4] = "High";
scoreMap[5] = "High";

var pipeline = mlContext.Transforms.Conversion.MapValue(
    "TimeframeCategory", timeframeMap, "Timeframe").Append(mlContext.
    Transforms.Conversion.MapValue("ScoreCategory", scoreMap, "Score"))
    .Append(mlContext.Transforms.Conversion.MapValue("Label",
    timeframeKeyMap, "Timeframe", treatValuesAsKeyType: true));

In [ ]:
IDataView transformedData = pipeline.Fit(data).Transform(data);

In [ ]:
var features = mlContext.Data.CreateEnumerable<TransformedData>(transformedData, reuseRowObject: false);

Console.WriteLine(" Timeframe   TimeframeCategory   Label    Score   ScoreCategory");

foreach (var featureRow in features)
{
    Console.WriteLine($"{featureRow.Timeframe}\t\t{featureRow.TimeframeCategory}\t\t\t{featureRow.Label}\t\t{featureRow.Score}\t{featureRow.ScoreCategory}");
}

public class TransformedData : DataPoint
{
    public string TimeframeCategory { get; set; }
    public string ScoreCategory { get; set; }
    public uint Label { get; set; }
}